<a href="https://colab.research.google.com/github/dohyeon-kim012/WebScraping/blob/main/%EC%8B%A4%EC%8A%B5_%EB%8B%A4%EC%9D%8C%20%EB%89%B4%EC%8A%A4%20%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 주제  
: `daum` 뉴스에서 분류 카테고리와 기사 제목들을 스크레이핑 하여 기사제목들이 알맞게 분류되는지 모델링  
  
- 카테고리 : `사회`, `정치`, `경제`, `국제`, `문화`, `IT` - 총 **6개**

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import requests
import json
import pandas as pd

# 전체 카테고리 크롤링

https://news.daum.net/society#1

- 카테고리명

https://news.daum.net/api/harmonydic/contents/news.json?category=society&approved=true&page=2&pageSize=20&pagesToShow=10&range=1

- 기사 제목
- 기사 내용

### 카테고리명 추출

In [ ]:
# 1. 카테고리명 추출
url_society = 'https://news.daum.net/society#1'

response_society = requests.get(url_society).content
data_society = BeautifulSoup(response_society, 'html.parser')
data_society


<!DOCTYPE html>

<html class="os_unknown none unknown version_0 " lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="always" name="referrer">
<meta content="Daum 뉴스" property="og:author"/>
<meta content="다음뉴스" property="og:site_name"/>
<meta content="사회" property="og:title"/>
<meta content="https://t1.daumcdn.net/media/img-media/mobile/meta/news.png" property="og:image"/>
<meta content="다음뉴스" property="og:description"/>
<meta content="https://news.daum.net/society" property="og:url"/>
<link href="https://m2.daumcdn.net/img-media/2010ci/Daum_favicon.ico" rel="shortcut icon"/>
<title>사회 | 다음뉴스</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="//t1.daumcdn.net/media/kraken/news/7031c97/common.css.merged.css" rel="stylesheet" type="text/css">
<link href="//t1.daumcdn.net/media/kraken/news/7031c97/news.css.merged.css" rel="stylesheet" type="text/css"/>
<link href="//t1.daumcdn.net/media/kraken/news/7031c97/calendar.css.merged.css" rel="stylesheet" type="text/

In [ ]:
# 2. 먼저 '사회'면만 잘 나오는지 확인
category_society = data_society.select_one("#kakaoGnb > div > ul > li.on > a > span.ir_wa").text
category_society

'사회'

In [ ]:
total_category = data_society.select_one("#kakaoGnb > div > ul")
all_category = total_category.select("li")
len(all_category)

11

In [ ]:
# 3. 목표로 하는 `사회 ~ IT`까지만 뽑아내기 위해 for문으로 list에 넣어줌
category_list = []

for categories in all_category[1:7]:
    category = categories.text
    category_list.append(category)

In [ ]:
category_list

[' 사회', '정치', '경제', '국제', '문화', 'IT']

원하는 모든 카테고리명을 한 개의 list에 넣어 두었음  
이후 DataFrame을 만들 때 사용할 예정

---

## 기사제목 및 기사내용 크롤링

Request URL: ''https://news.daum.net/breakingnews/society?page=1&regDate=20210710'  

위 URL에서 가장 중요한 파라미터는 **`category`** 와 **`page`** 이므로,  
두 개의 파라미터만 값을 따로 지정해줄 수 있는지 시도해 볼 예정  

기사의 개수는 한 페이지 당 15 개씩이며, 충분한 데이터 확보를 위해 카테고리 당 100페이지까지 크롤링 할 예정

# 사회 카테고리 1페이지 20210710일 기사 크롤링 확인

In [ ]:
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup

## url requests

In [ ]:
url = 'https://news.daum.net/breakingnews/society?page=1&regDate=20210710'
res = requests.get(url)
res

<Response [200]>

## bs 객체로 변환

In [ ]:
soup = BeautifulSoup(res.content)
soup

<!DOCTYPE html>
<html class="os_unknown none unknown version_0 " lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="always" name="referrer"/>
<meta content="Daum 뉴스" property="og:author"/>
<meta content="다음뉴스" property="og:site_name"/>
<meta content="전체뉴스" property="og:title"/>
<meta content="https://t1.daumcdn.net/media/img-media/mobile/meta/news.png" property="og:image"/>
<meta content="다음뉴스" property="og:description"/>
<meta content="https://news.daum.net/breakingnews/society" property="og:url"/>
<link href="https://m2.daumcdn.net/img-media/2010ci/Daum_favicon.ico" rel="shortcut icon"/>
<title>전체뉴스 | 다음뉴스</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="//t1.daumcdn.net/media/kraken/news/7031c97/common.css.merged.css" rel="stylesheet" type="text/css"/>
<link href="//t1.daumcdn.net/media/kraken/news/7031c97/news.css.merged.css" rel="stylesheet" type="text/css"/>
<link href="//t1.daumcdn.net/media/kraken/news/7031c97/calendar.css.merged.css" rel="styles

## 페이지별 각 기사 링크를 가져오기

In [ ]:
url_list = soup.select("#mArticle > div.box_etc > ul > li > a")
url_list

[<a class="link_thumb" href="https://v.daum.net/v/20210710235142699">
 <img alt="만취 40대 몰던 트럭, 오토바이 들이받아..1명 부상" class="thumb_g" src="https://img1.daumcdn.net/thumb/S95x77ht.u/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fnews%2F202107%2F10%2Fnewsis%2F20210710235142765lhfx.jpg&amp;scode=media"/>
 </a>, <a class="link_thumb" href="https://v.daum.net/v/20210710234601660">
 <img alt='이재명, 하루만에 후원금 9억.."희망민국 바꿀 종자돈, 감사하다"' class="thumb_g" src="https://img1.daumcdn.net/thumb/S95x77ht.u/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fnews%2F202107%2F10%2Fseoul%2F20210710234601374tyhx.jpg&amp;scode=media"/>
 </a>, <a class="link_thumb" href="https://v.daum.net/v/20210710234534654">
 <img alt='서울대 학생처장, "피해자 코스프레 역겹다고 한 건 이재명 글 보고.." 해명' class="thumb_g" src="https://img1.daumcdn.net/thumb/S95x77ht.u/?fname=https%3A%2F%2Ft1.daumcdn.net%2Fnews%2F202107%2F10%2Fsegye%2F20210710234534970rmhs.jpg&amp;scode=media"/>
 </a>, <a class="link_thumb" href="https://v.daum.net/v/20210710232432527">
 <img alt="파주시 6회연속 지방재정 신속

In [ ]:
url_list[0]['href']

'https://v.daum.net/v/20210710235142699'

## 페이지별 기사 개수 확인

In [ ]:
len(url_list)

13

## 각 기사 url을 통해 내부 기사로 들어가기

In [ ]:
res = requests.get(url_list[0]['href'])
res

<Response [200]>

In [ ]:
soup = BeautifulSoup(res.content)
soup

<!DOCTYPE html>
<html lang="ko">
<head data-cloud-area="head">
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<style>
            @import url('//t1.daumcdn.net/harmony_static/cloud/page/a3807badcf782bfe521bceb8b042040ee11676b0.css');
            @import url('//t1.daumcdn.net/harmony_static/cloud/2021/07/09/common.08ed249def6c63782e5d.css')
        </style>
<style>
            /* 다음 카페 소셜 쉐어 임시 비활성화 */
            .list_sns li:nth-child(6) {
                display: none;
            }
        </style>
<meta content="k5M0e22kuh" property="mccp:docId"/>
<meta content="다음뉴스" property="og:site_name"/>
<meta content="만취 40대 몰던 트럭, 오토바이 들이받아..1명 부상" property="og:title"/>
<meta content="20210710235142" property="og:regDate"/>
<meta content="article" property="og:type"/>
<meta content="뉴시스" property="og:article:author"/>
<meta content="https://news.v.daum.net/v/k5M0e22kuh" property="og:url"/>
<meta content="https://img1.daumcdn.net/thumb/S1200x630/?fname=https:/

## 타이틀 가져오기

In [ ]:
soup.select_one('#cSub > div > h3').text

'만취 40대 몰던 트럭, 오토바이 들이받아..1명 부상'

## 기사 내용 가져오기

In [ ]:
content = soup.select_one('#harmonyContainer > section').text.strip()
content

'[광주=뉴시스] 변재훈 기자 = 광주 도심에서 술에 취한 40대가 화물차를 몰다가 오토바이를 들이받았다.\n10일 광주 광산경찰서에 따르면, 이날 오후 10시22분께 광산구 월곡동 한 교차로에서 A(42)씨가 몰던 1t 트럭이 B(33)씨의 오토바이를 들이받았다.\n이 사고로 오토바이 운전자가 다쳐 인근 병원으로 옮겨져 치료를 받고 있다.\n조사 결과 화물차 운전자 A씨의 혈중알코올농도는 0.077%로 측정돼 운전 면허 정지 수치에 해당하는 만취 상태였던 것으로 드러났다.\n경찰은 A씨를 도로교통법 위반 등 혐의로 입건, 정확한 사건 경위를 조사하고 있다.\n☞공감언론 뉴시스 wisdom21@newsis.com'

# 최종 스크레이퍼
- get방식
- category: 카테고리, page: 페이지, regDate: 날짜

In [ ]:
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup

# 두 개의 url에 각각 변수를 넣어줘서 총 50페이지(페이지당 15개)의 기사 url을 list로 만들어둠
# category
category_list = ['society', 'politics', 'economic', 'foreign', 'culture', 'digital']
# 데이터 프레임을 만들 딕셔너리 생성
df_news = {
    "category" : [],
    "news_url" : [],
    "title" : [],
    "content" : [],
}

url_params = 'https://news.daum.net/breakingnews/{}?page={}&regDate={}'
date = '20210520'
newsId_list = []
# 카테고리를 society -> politics -> ... -> digital 순으로 반복
for category in tqdm(category_list):
  # 페이지를 1 -> 2 -> ... -> 30페이지 순으로 반복
  for page in tqdm(range(1, 51, 5)): 
    #url_list : category별 url 주소 모음
    url_list = []
    # page_url: 카테고리와 페이지 정보를 넣은 url
    page_url = url_params.format(category, page, date)
    # page_res: 해당 url을 요청 
    page_res = requests.get(page_url).content
    # page_html: 해당 url의 BeautifulSoup으로 html 정보를 가져옴
    page_html = BeautifulSoup(page_res, 'html.parser')
    idxs = len(page_html.select("#mArticle > div.box_etc > ul > li > a"))
    # print(page_html)
    for idx in range(idxs):
        content_url = page_html.select("#mArticle > div.box_etc > ul > li > a")[idx]["href"]
        url_list.append(content_url)
    # idxs: html의 list키에 각각 기사에 대한 정보가 있기 때문에 기사의 index range형태로 가져오기
    # idxs = range(len(content_url))
    # 해당하는 페이지의 기사 개수만큼 반복(20)
    for url in set(url_list):
        # page_info = page_html['list'][idx]
        # 카테고리 추가
        df_news['category'].append(category)
        # url 추가
        df_news['news_url'].append(url)
        # content_res: 현재 idx의 url을 요청
        content_res = requests.get(url).content
        soup = BeautifulSoup(content_res)
        # title 추가
        title = soup.select_one('#cSub > div > h3').text
        df_news['title'].append(title)
        # content추가
        content = soup.select_one('#harmonyContainer > section').text.strip()
        df_news['content'].append(content)


# 페이지 구간별 df 만들어 합치기

In [ ]:
import pandas as pd
df_0520 = pd.DataFrame(df_news)
df_0520

,category,news_url,title,content
0,society,https://v.daum.net/v/20210520234807470,"""'붙이는 소화기'로 전통시장 화재 막아요""",[KBS 울산] [앵커]\n전통시장 화재는 한 번 불이 나면 큰 피해로 이어지는 경...
1,society,https://v.daum.net/v/20210520234804469,"""'화물차 안전운임제' 위반에도 처벌 없어""",[KBS 울산]화물연대 울산본부는 울산시청 앞에서 기자회견을 열고 장시간 운행 금지...
2,society,https://v.daum.net/v/20210520234800466,"""산단 주민 암발생률 높아..유해물질과 연관""",[KBS 울산] [앵커]\n최근 환경부의 울산 산업단지 건강피해 예비타당성 조사 결...
3,society,https://v.daum.net/v/20210520234802467,"현대重 노조, 산재 관련 회사 대표 등 12명 고발",[KBS 울산]지난 8일 발생한 현대중공업 하청업체 직원의 산재 사망사고와 관련해 ...
4,society,https://v.daum.net/v/20210520235557562,"화이자, 1차 때보다 2차 접종 때 이상반응 더 흔해",[앵커]\n현재 코로나19 백신 예방접종 사전 예약이 진행 중인 가운데 오는 27일...
...,...,...,...,...
844,digital,https://v.daum.net/v/20210520103730815,"NHN KCP, 페이오니아와 '글로벌 마켓플레이스 진출 위한 포럼'",NHN한국사이버결제는 페이오니아와 함께 지난 18일 `글로벌 마켓플레이스 진출을 위...
845,digital,https://v.daum.net/v/20210520103819849,거대한 붉은 달 '슈퍼블러드문'..3년만에 다시 뜬다,31일 오후 서울 롯데월드타워 너머로 슈퍼문이 개기월식을 시작하고 있다. 이번 개기...
846,digital,https://v.daum.net/v/20210520104100971,"와디즈, 한국 색 담은 메이커 지원",(지디넷코리아=백봉삼 기자)와디즈(대표 신혜성)는 한국 전통의 것을 보존하고 새로운...
847,digital,https://v.daum.net/v/20210520103903881,"KT엠모바일, 네이버 쇼핑 라이브서 유심 '100원'",KT엠모바일이 100원 유심 네이버 쇼핑 라이브를 진행한다.\n ...


In [ ]:
df_0520.nunique()

category      6
news_url    849
title       740
content     805
dtype: int64

In [ ]:
df_0530 = pd.DataFrame(df_news)
df_0530

,category,news_url,title,content
0,society,https://v.daum.net/v/20210530234359628,"선문대, 간호학과 대상 가상현실 융합 실습 교육 진행",선문대학교는 최근 간호학과의 가상현실 융합 실습 교육을 실시했다고 30일 밝혔다.\...
1,society,https://v.daum.net/v/20210530233305561,경기도-경기북부 도의원 지역균형발전 해법모색,경기도-경기북부도의원협의회 28일 정책간담회 개최. 사진제공=경기북부청\n ...
2,society,https://v.daum.net/v/20210530233711593,美가 우리 군에 제공한 얀센 백신..혈전증 우려로 30세 이상 예비군·민방위 등에 ...,정은경 코로나19 예방접종대응추진단장(질병관리청장)이 30일 종로구 세종대로 정부 ...
3,society,https://v.daum.net/v/20210530233442569,[Q플러스] 뉴스에 반(反)하자,김솔희 : 흔히 미디어를 세상을 보는 창으로 비유합니다. 창을 통해 바깥세상을 볼 ...
4,society,https://v.daum.net/v/20210530233751600,[윤종빈칼럼] '이준석 돌풍' 숨은 진실은,국민의힘 당대표 예비경선에서 만 36세인 ‘0’선의 이준석 전 최고위원이 1위를 차...
...,...,...,...,...
743,digital,https://v.daum.net/v/20210530041611095,바르면 표면 온도 '뚝'..복사냉각 페인트 개발,[앵커] \n발라만 주면 표면 온도를 최대 10도가량 낮춰주는 페인트가 개발됐습니다...
744,digital,https://v.daum.net/v/20210530041611095,바르면 표면 온도 '뚝'..복사냉각 페인트 개발,[앵커] \n발라만 주면 표면 온도를 최대 10도가량 낮춰주는 페인트가 개발됐습니다...
745,digital,https://v.daum.net/v/20210530041611095,바르면 표면 온도 '뚝'..복사냉각 페인트 개발,[앵커] \n발라만 주면 표면 온도를 최대 10도가량 낮춰주는 페인트가 개발됐습니다...
746,digital,https://v.daum.net/v/20210530041611095,바르면 표면 온도 '뚝'..복사냉각 페인트 개발,[앵커] \n발라만 주면 표면 온도를 최대 10도가량 낮춰주는 페인트가 개발됐습니다...


In [ ]:
df_0530.nunique()

NameError: ignored

In [ ]:
df_0701_3 = pd.DataFrame(df_news)
df_0701_3

In [ ]:
df_0701_3.nunique()

In [ ]:
df_2105 = pd.concat([df_0701_1, df_0701_2, df_0701_3], ignore_index=True)
df_2105


# 만들어진 df 저장하기

In [ ]:
df_0701.to_csv('df_20210701.csv', index=False)

# 저장된 df 제대로 불러와지는지 확인하기

In [ ]:
df = pd.read_csv('df_20210701.csv')
df

In [ ]:
df[df['category'] == 'digital']